In [1]:
import bs4
import requests as r
from IPython.display import Javascript
import pprint
import pandas as pd
from datetime import date

def open_web(URL):
    display(Javascript('window.open("{url}");'.format(url=URL)))

In [2]:
PAGE = 1
URL = "https://www.strumentimusicali.net/advanced_search_result.php?sort=3d&categories_id=21&inc_subcat=1&keywords=&page=" + str(PAGE) + "&products_available=1&used=1"

URL_prefix = "https://www.strumentimusicali.net/product_info.php/products_id/"

response = r.get(URL)
response.raise_for_status()
soup = bs4.BeautifulSoup(response.text, "html.parser")
annunci = soup.find_all('tr', class_='productListing-even')

names = []
prices = []
links = []

while (len(annunci) > 0):

    for annuncio in annunci:

        title = annuncio.find('b').text.strip()
        link = annuncio.find('a')['href']
        price = annuncio.find('span', class_='d-block fontSize14 marginBottom5 bold')

        if price is None: # sconto
            price = annuncio.find('span', class_='productSpecialPrice bold')
        price = price.text

        if URL_prefix in link:
            names.append(title)
            prices.append(price)
            links.append(link)

    PAGE += 1
    URL = "https://www.strumentimusicali.net/advanced_search_result.php?sort=3d&categories_id=21&inc_subcat=1&keywords=&page=" + str(PAGE) + "&products_available=1&used=1"
    response = r.get(URL)
    response.raise_for_status()
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    annunci = soup.find_all('tr', class_='productListing-even')


products = {'Titolo Annuncio': names,
        'Prezzo': prices,
        'Link': links
        }
df = pd.DataFrame(products)
df.to_csv("usato"+str(date.today())+".csv", index=False)